<h1 style="font-family:Impact,Arial;font-size:30px;">37004 Interest Rates and Credit Risk Models - Spring 2024</h1>
<h1 style="font-family:Impact,Arial;font-size:45px;">Assignment Part 2</h1>
<h2 style="font-family:Arial;">Erik Schl&ouml;gl</h2>
<p><small> School of Mathematical &amp; Physical Sciences<br>
University of Technology Sydney
</small></p>
<p>
<a href="mailto:Erik.Schlogl@uts.edu.au?Subject=37000 JIT" target="_blank">
<small><font color=MediumVioletRed>Erik.Schlogl@uts.edu.au</font></small></a>
</p>
<hr style="height:5px;border:none;color:#333;background-color:#333;" />

Consider the data files <code>daily-treasury-rates2018.csv</code>, <code>daily-treasury-rates2019.csv</code>, <code>daily-treasury-rates2020.csv</code>, <code>daily-treasury-rates2021.csv</code> and <code>daily-treasury-rates2022.csv</code>. They contain the United States Treasury's "Daily Yield Curve" data (sourced from www.treasury.gov) for the years 2018 to 2022. These rates are what the US Treasury calls "CMT rates" (see
https://www.treasury.gov/resource-center/faqs/Interest-Rates/Pages/faq.aspx).

In [9]:
import pandas as pd
import numpy as np

In [10]:
# Load and concatenate the treasury data for the years 2018-2022
data_frames = []
for year in range(18, 23):
    data_frames.append(pd.read_csv(f'./daily-treasury-rates20{year}.csv'))

treasury_data = pd.concat(data_frames).reset_index(drop=True)


<H2>Task 1:</H2>
Convert these rates into continuously compounded yields. <I>(1 mark)</I>

In [11]:
# Convert discrete yields to continuously compounded yields
maturity_cols = treasury_data.columns[1:]
for col in maturity_cols:
    treasury_data[col] = np.log(1 + treasury_data[col] / 100)

# Display the first few rows to check the updated dataset
treasury_data.head()

,Date,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,12/31/2018,0.024107,0.024205,0.024205,0.025278,0.025960,0.024497,0.024302,0.024790,0.025570,0.026545,0.028296,0.029753
1,12/28/2018,0.023619,0.024400,0.023717,0.024497,0.025375,0.024888,0.024693,0.025278,0.025960,0.026837,0.028490,0.029947
2,12/27/2018,0.024009,0.024302,0.023814,0.024595,0.025473,0.025278,0.025180,0.025668,0.026447,0.027323,0.028782,0.030044
3,12/26/2018,0.023814,0.024009,0.024107,0.025083,0.025765,0.025765,0.025668,0.026350,0.027031,0.027712,0.028976,0.030141
4,12/24/2018,0.023912,0.024009,0.024205,0.024888,0.025765,0.025180,0.025278,0.025473,0.026252,0.027031,0.028393,0.029559


In this data set, the <I>time to maturity</I> (not the time of maturity) is constant in each column. Thus, from the data in each column, we can obtain an estimate of the variance of the day-to-day changes in a continuously compounded yield with a given time to maturity. In order to relate this back to an HJM model, we need the dynamics of rates with fixed time to maturity in the model. 

We start with the instantaneous forward rates. The instantaneous forward rates $f(t,T)$ modelled in HJM have a fixed time of maturity $T$, with dynamics
$$
f(t,T)=f(0,T)+\int_0^t\sigma(u,T)\int_u^T\sigma(u,s)dsdu+\int_0^t\sigma(u,T)dW_{\beta}(u)\quad\forall\
t\in[0,T]
$$
Define instantaneous forward rates $r(t,x)$ with constant time to maturity $x$ by
$$
r(t,x)=f(t,t+x)
$$
The dynamics of $r(t,x)$ are given by the <I>Musiela equation</I> (for a proof, see e.g. Section 18.3 of Bj&ouml;rk, T. (1998) <I>Arbitrage Theory in Continuous Time</I>, Oxford University Press)
$$
r(t,x)=r(0,x)+\int_0^t\left(\frac{\partial}{\partial x}r(u,x)+\sigma(u,u+x)\int_0^x\sigma(u,u+s)ds\right)du+\int_0^t\sigma(u,u+x)dW_{\beta}(u)
$$

The continously compounded yield with constant time to maturity $x$ is
$$
y(t,x)=\frac1x\int_0^xr(t,s)ds
$$
Thus we have for the change in the continously compounded yield with constant time to maturity over a $\Delta t$ time step:
$$
y(t+\Delta t,x)-y(t,x)=\frac1x\int_0^x\left(\int_t^{t+\Delta t}\left(\frac{\partial}{\partial s}r(u,s)+\sigma(u,u+s)\int_0^s\sigma(u,u+v)dv\right)du+\int_t^{t+\Delta t}\sigma(u,u+s)dW_{\beta}(u)\right)ds
$$
Note that the drift will contribute to the variance of changes in the continously compounded yield via
$$
\frac{\partial}{\partial s}r(u,s),
$$
but for practical purposes this is negligible. Thus
\begin{eqnarray*}
\text{Var}[y(t+\Delta t,x)-y(t,x)] &\approx& \text{Var}\left[\frac1x\int_0^x\int_t^{t+\Delta t}\sigma(u,u+s)dW_{\beta}(u)ds\right]\\
&=& \text{Var}\left[\frac1x\int_t^{t+\Delta t}\int_0^x\nu e^{-a(u+s-u)}dsdW_{\beta}(u)\right]\\
&=& \text{Var}\left[\frac1x\int_t^{t+\Delta t}\frac{\nu}a(1-e^{-ax})dW_{\beta}(u)\right]\\
&=& \left(\frac1x\frac{\nu}a(1-e^{-ax})\right)^2\Delta t
\end{eqnarray*}

<H2>Task 2:</H2>
Assuming a one-factor Gauss/Markov HJM model with mean reversion fixed at $a = 0.1$, use all the data for the six-month yields to determine an appropriate choice of the volatility level $\nu$ (notation as above). For simplicity, you may assume that the data observations are equally spaced in calendar time. <I>(3 marks)</I>

In [12]:
# Calculate daily changes for the 6-month yield and variance
treasury_data['6Mo_diff'] = treasury_data['6 Mo'].diff()
var_6mo = treasury_data['6Mo_diff'].var()

# Given parameters for calculation
a_param = 0.1  # mean reversion parameter
x_maturity = 0.5  # 6-month maturity (in years)

# Solve for nu using the derived variance and formula
nu_value = a_param * (x_maturity * np.sqrt(var_6mo)) / (1 - np.exp(-a_param * x_maturity))
nu_value


0.0012989977652025032

<H2>Task 3:</H2>
Assuming a one-factor Gauss/Markov HJM model, use all the data for the one-month and thirty-year yields to determine an appropriate choice of the mean reversion parameter $a$ and volatility level $\nu$. For simplicity, you may assume that the data observations are equally spaced in calendar time. <I>(3 marks)</I>

In [16]:
# Calculate daily changes for the 1-month and 30-year yields
treasury_data['1Mo_diff'] = treasury_data['1 Mo'].diff()
treasury_data['30Yr_diff'] = treasury_data['30 Yr'].diff()

# Compute variances for 1-month and 30-year yields
var_1mo = treasury_data['1Mo_diff'].var()
var_30yr = treasury_data['30Yr_diff'].var()
var_1mo,var_30yr

(1.2217867621507136e-06, 6.389898152370723e-07)

In [17]:
# Solve system of equations for a and nu using fsolve
from scipy.optimize import fsolve

def eqns(params):
    a_val, nu_val = params
    eq1 = (12 * nu_val / a_val * (1 - np.exp(-a_val / 12)))**2 - var_1mo
    eq2 = (1 / 30 * nu_val / a_val * (1 - np.exp(-30 * a_val)))**2 - var_30yr
    return [eq1, eq2]

# Initial guesses for parameters
initial_guess = [0.1, 0.001]

# Solve the equations
a_solution, nu_solution = fsolve(eqns, initial_guess)

a_solution, nu_solution

(0.022985515614413112, 0.001106403594485407)

A way in which the performance of a model for risk management purposes can be <B>backtested</B> is to use historical interest data to analyse what would have happened if one had used the model to hedge a derivative security in the past. This motivates the following Task:

<H2>Task 4:</H2>
Suppose that at the beginning of January 2022 you have sold an at-the-money European receiver swaption, expiring in six months, where the frequency of payments of the underlying swap is quarterly and the swap ends five years after the swaption expiry. If you hedge this swaption with rebalancing on every day for which there is data, using the natural hedge instruments of the constituent zero coupon bond options, what would be your profit/loss when this option matures? Assume that swaption price and hedges are calculated using a one-factor Gauss/Markov HJM model with the parameters determined as in Task 3, but <B>using only data which would have been available at the time the swaption was sold</B>. Assume that any profit/loss is invested/borrowed by buying/selling the zero coupon bond maturing at the same time as the swaption. Use loglinear interpolation of zero coupon bond prices, but only where necessary. Repeat
this for a (otherwise identical) swaption sold at the beginning of, respectively, April 2021, July 2021, and October 2021. Discuss the possible causes of profit/loss. <I>(13 marks)</I>

In [18]:
# Function for log-linear interpolation of zero-coupon bond prices
def log_interp(ylds, times, target):
    """ Perform log-linear interpolation on yields to estimate bond prices. """
    log_ylds = np.log(ylds)
    interp_log_yld = np.interp(target, times, log_ylds)
    return np.exp(-interp_log_yld * target)

# Example maturities and yields from the dataset for testing
maturs = [0.5, 1, 2, 5, 10, 30]
yields_ex = treasury_data[['6 Mo', '1 Yr', '2 Yr', '5 Yr', '10 Yr', '30 Yr']].iloc[-1].values

# Example of interpolating bond price for 7-year maturity
target_matur = 7
interpolated_price = log_interp(yields_ex, maturs, target_matur)
interpolated_price


7143584338117.364

In [19]:
# Function to calculate bond price using interpolated continuously compounded yields
def bond_price(yld, t):
    """ Calculate zero-coupon bond price given continuously compounded yield. """
    return np.exp(-yld * t)

# Log-linear interpolation for continuously compounded yields
def log_yield_interp(ylds, maturs, target):
    """ Interpolate yields and compute bond prices based on interpolated yield. """
    log_ylds = np.log(1 + ylds)
    interp_log_yld = np.interp(target, maturs, log_ylds)
    cont_yld = np.exp(interp_log_yld) - 1
    return bond_price(cont_yld, target)

# Test with corrected interpolation method for a 7-year bond
corrected_bond_price = log_yield_interp(yields_ex, maturs, target_matur)
corrected_bond_price


0.9026539595216653